<a href="https://colab.research.google.com/github/biaxz/recap/blob/main/recap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Auto Movie Recap Editor

This notebook creates auto-edited movie recaps with random effects including:
- Slow motion (25%)
- Freeze frames (25%)
- Normal speed (50%)
- Transitions
- Multiple timestamp exclusions for opening/credit scenes
- Sequential segments from start to end

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Install required packages
!pip install moviepy numpy

In [ ]:
from moviepy.editor import VideoFileClip, AudioFileClip, concatenate_videoclips, vfx
import numpy as np
import random

# File paths
movie_path = "/content/drive/MyDrive/EDIT/Lk21.DE-Venom-The-Last-Dance-2024-BluRay-1736553019.mp4"
audio_path = "/content/drive/MyDrive/EDIT/mix_14m56s (audio-joiner.com).mp3"

# Define excluded timestamps (start_time, end_time) in seconds
excluded_timestamps = [
    (0, 41), #in seconds        # Example: Skip 00:00:00 to 00:00:41
    (129, 153), #in seconds     # Example: Skip 00:00:30 to 00:00:40 (opening)
    (5549, 5678), #in seconds  # Example: Skip 01:00:30 to 01:00:40 (credits)
    (5704, 6486) #in seconds
]

def is_timestamp_excluded(time):
    """Check if a timestamp falls within any excluded ranges"""
    return any(start <= time <= end for start, end in excluded_timestamps)

def get_valid_timestamp(start_range, end_range, duration):
    """Get a valid timestamp that's not in excluded ranges"""
    max_attempts = 50  # Prevent infinite loop
    attempts = 0

    while attempts < max_attempts:
        time = random.uniform(start_range, min(end_range, duration))
        # Check if this timestamp and the next few seconds are valid
        if not any(is_timestamp_excluded(t) for t in np.arange(time, time + 4, 0.5)):
            return time
        attempts += 1

    raise ValueError("Could not find valid timestamp after maximum attempts")

def apply_fade_transition(clip, duration):
    """Apply fade in/out transition to a clip"""
    return clip.fadein(0.5).fadeout(0.5)

def create_random_effect_clip(clip, start_time, duration, effect_type, apply_transition=False):
    subclip = clip.subclip(start_time, start_time + duration)

    # Apply the main effect
    if effect_type == "slowmo":
        effect_clip = subclip.fx(vfx.speedx, 0.5)
    elif effect_type == "freeze":
        effect_clip = subclip.to_ImageClip(t=0).set_duration(duration)
    else:  # normal speed
        effect_clip = subclip

    # Apply transition if requested
    if apply_transition:
        effect_clip = apply_fade_transition(effect_clip, duration)

    return effect_clip

def auto_edit_video(movie_path, audio_path):
    # Load video and audio
    video = VideoFileClip(movie_path)
    audio = AudioFileClip(audio_path)

    # Get target duration from audio
    target_duration = audio.duration

    # Calculate number of segments needed (using 3 seconds as average duration)
    avg_segment_duration = 3  # seconds
    total_segments = int(target_duration / avg_segment_duration)

    # Calculate segments distribution
    sequential_segments = int(total_segments * 0.65)  # 65% sequential
    random_segments = total_segments - sequential_segments  # 35% random

    # Calculate sequential step size
    sequential_duration = video.duration * 0.65  # 65% of total video
    sequential_step = sequential_duration / sequential_segments

    sequential_clips = []
    random_clips = []

    # Create sequential segments (65%) starting from 0 seconds
    sequential_start = 0  # Start from beginning
    for i in range(sequential_segments):
        segment_duration = random.uniform(2, 4)

        if sequential_start + segment_duration > video.duration:
            break

        # Skip if current segment overlaps with excluded timestamps
        if any(is_timestamp_excluded(t) for t in np.arange(sequential_start, sequential_start + segment_duration, 0.5)):
            sequential_start += sequential_step
            continue

        effect = random.choices(
            ["slowmo", "freeze", "normal"],
            weights=[0.25, 0.25, 0.50]
        )[0]

        apply_transition = random.random() < 0.5

        try:
            effect_clip = create_random_effect_clip(
                video,
                sequential_start,
                segment_duration,
                effect,
                apply_transition
            )

            sequential_clips.append(effect_clip)
            sequential_start += sequential_step
        except Exception as e:
            print(f"Error creating sequential segment {i}: {e}")
            sequential_start += sequential_step
            continue

    # Add random segments (35%)
    for i in range(random_segments):
        segment_duration = random.uniform(2, 4)

        try:
            # Get valid random start time
            random_start = get_valid_timestamp(0, video.duration - segment_duration, video.duration)

            effect = random.choices(
                ["slowmo", "freeze", "normal"],
                weights=[0.35, 0.25, 0.35]
            )[0]

            apply_transition = random.random() < 0.5

            effect_clip = create_random_effect_clip(
                video,
                random_start,
                segment_duration,
                effect,
                apply_transition
            )

            random_clips.append(effect_clip)
        except Exception as e:
            print(f"Error creating random segment {i}: {e}")
            continue

    # Shuffle the random segments
    random.shuffle(random_clips)

    # Combine all segments in order: sequential then random
    all_segments = sequential_clips + random_clips

    # Concatenate all segments
    final_video = concatenate_videoclips(all_segments)

    # Adjust final duration to match audio duration
    if final_video.duration > target_duration:
        final_video = final_video.subclip(0, target_duration)

    # Set audio
    final_video = final_video.set_audio(audio)

    return final_video

# Create and save the final video
try:
    print("Starting video editing...")
    final_video = auto_edit_video(movie_path, audio_path)

    output_path = "/content/drive/MyDrive/EDIT/output_recap_4.mp4"
    print(f"Saving video to {output_path}...")

    final_video.write_videofile(
        output_path,
        codec='libx264',
        audio_codec='aac',
        threads=4
    )

    print("Video editing completed successfully!")
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    # Clean up
    try:
        video.close()
        audio.close()
        final_video.close()
    except:
        pass

## How to Use

1. Mount your Google Drive
2. Update the `movie_path` and `audio_path` variables to point to your files
3. Define `excluded_timestamps` list with tuples of (start_time, end_time) in seconds
   Example: `[(30, 40), (3630, 3640)]` excludes 00:00:30-00:00:40 and 01:00:30-01:00:40
4. Run all cells
5. The output will be saved as 'output_recap.mp4' in your specified directory

The script will:
- Process sequential segments (65%) from the start of the video
- Skip all specified timestamp ranges (opening/credits/any unwanted scenes)
- Take 35% random segments from throughout the video (excluding specified ranges)
- Apply effects (slow motion, freeze frames, normal speed) with specified probabilities
- Add fade transitions to 50% of all clips
- Match the final video duration to the audio duration
- Preserve audio quality using AAC codec